In [7]:
import pandas as pd
from processing import clean_text, convert_emojis
import numpy as np 
import pandas as pd
import re
import emoji
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import string
from sklearn.preprocessing import LabelEncoder
import nltk
nltk.download('wordnet')
import tensorflow as tf
import pickle
import matplotlib.pyplot as plt
from underthesea import word_tokenize
lookup_dict = {'sp':'sản phẩm', 'dt':'điện thoại',"đt" : "điện thoại", "ko" : "không", "seal" : "tem_mới", "gút chóp" : "tốt", "m" : "mình", "ok" : "tốt", "chẹp" : "đẹp",
               "đc" : "được","dc" : "được", "chất" : "tốt","gudddddddddddddddddddd":"tốt", "vs" : "với", "ng" : "người", "triệu like":"quá tốt", "feedback":"phản hồi",
               "ngon lành cành đào":"quá tốt", "dag":"đang","qc":"quảng cáo","k" : "không","hd sd": "hướng dẫn sử dụng","sd" : "sử dụng","hd": "hướng dẫn",
               "nt":"nhắn tin"}

chars_regrex = '[aàảãáạăằẳẵắặâầẩẫấậAÀẢÃÁẠĂẰẲẴẮẶÂẦẨẪẤẬoòỏõóọôồổỗốộơờởỡớợOÒỎÕÓỌÔỒỔỖỐỘƠỜỞỠỚỢeèẻẽéẹêềểễếệEÈẺẼÉẸÊỀỂỄẾỆuùủũúụưừửữứựUÙỦŨÚỤƯỪỬỮỨỰiìỉĩíịIÌỈĨÍỊyỳỷỹýỵYỲỶỸÝỴnNvVmMCG]'
same_chars = {
    'a': ['á', 'à', 'ả', 'ã', 'ạ', 'ấ', 'ầ', 'ẩ', 'ẫ', 'ậ', 'ắ', 'ằ', 'ẳ', 'ẵ', 'ặ'],
    'A': ['Á','À','Ả','Ã','Ạ','Ấ','Ầ','Ẩ','Ẫ','Ậ','Ắ','Ằ','Ẳ','Ẵ','Ặ'],
    'O': ['Ó','Ò','Ỏ','Õ','Ọ','Ô','Ố','Ồ','Ổ','Ỗ','Ộ','Ơ','Ớ','Ờ','Ở','Ỡ','Ợ','Q'],
    'o': ['ó', 'ò', 'ỏ', 'õ', 'ọ', 'ô', 'ố', 'ồ', 'ổ', 'ỗ', 'ộ', 'ơ','ớ', 'ờ', 'ở', 'ỡ', 'ợ', 'q'],
    'e': ['é', 'è', 'ẻ', 'ẽ', 'ẹ', 'ế', 'ề', 'ể', 'ễ', 'ệ', 'ê'],
    'E': ['É', 'È', 'Ẻ', 'Ẽ', 'Ẹ', 'Ế', 'Ề', 'Ể', 'Ễ', 'Ệ', 'Ê'],
    'u': ['ú', 'ù', 'ủ', 'ũ', 'ụ', 'ứ', 'ừ', 'ử', 'ữ', 'ự', 'ư'],
    'U': ['Ú', 'Ù', 'Ủ', 'Ũ', 'Ụ', 'Ứ', 'Ừ', 'Ử', 'Ữ', 'Ự', 'Ư'],
    'i': ['í', 'ì', 'ỉ', 'ĩ', 'ị'],
    'I': ['Í', 'Ì', 'Ỉ', 'Ĩ', 'Ị'],
    'y': ['ý', 'ỳ', 'ỷ', 'ỹ', 'ỵ', 'v'],
    'Y': ['Ý', 'Ỳ', 'Ỷ', 'Ỹ', 'Ỵ', 'V'],
    'n': ['m'],
    'N': ['N'],
    'v': ['y'],
    'V': ['Y'],
    'm': ['n'],
    'M': ['N'],
    'C': ['G'],
    'G': ['C']
}


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\HELLO\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
data = pd.read_csv('Tiki_Smartphone.csv')

In [3]:
sentences = data['content'].values
labels = data['title'].values
labels_dict = {'Rất không hài lòng': 1, 'Không hài lòng':2,'Bình thường':3, 'Hài lòng':4, 'Cực kì hài lòng': 5}

In [4]:
sentences

array(['Nhân viên giao hàng nhanh chóng, điện thoại nhỏ nhắn cầm như hợp kem chống nắng vậy.điểm trừ là sạc pin hơi lâu,pin tụt nhanh như tụt quần.không thực sự mượt mà đâu,(máy mới mà)có vẻ hơi lag nhẹ.nhà bán hàng uy tín .',
       'Chụp đẹp , nhỏ gọn , máy rất đẹp và giá tốt', 'Hàng chất lương',
       ..., nan,
       'Bán dt bị hư mà cả 10 ngày ko giải quyết hoàn tiền lại cho khách hàng gì đâu mới mua mà đt giật màn hình rồi giờ ko giải quyết luôn',
       nan], dtype=object)

## Tiền xử lý

In [5]:
# Áp dụng hàm clean_text cho mỗi câu và thay thế trực tiếp trong cột 'content'
sentences = [clean_text(sentence, lookup_dict) for sentence in sentences]


In [6]:
sentences

['nhân_viên giao hàng nhanh_chóng điện_thoại nhỏ_nhắn cầm như hợp kem chống nắng vậy điểm trừ là sạc pin hơi lâu_pin tụt nhanh như tụt_quần không thực_sự mượt_mà đâu máy mới mà có_vẻ hơi lag nhẹ nhà bán hàng uy_tín',
 'chụp đẹp nhỏ gọn máy rất đẹp và giá tốt',
 'hàng tốt lương',
 'máy rất tốt nhưng lúc mua tôi quên không mua kèm_gói bảo_vệ màn_hình vậy giờ tôi có_thể mua lại gói này riêng không',
 'sản_phẩm tốt giao hàng nhanh tuy_nhiên shop chưa có mục chat mình muốn hỏi kích_hoạt bảo_hành thế_nào shop',
 nan,
 'giao hàng siêu nhanh điện_thoại đẹp đã dán màn_hình hàng chính hãng nguyên_tem_mới',
 'máy mới nguyên tem_mới',
 nan,
 'điện_thoại của tôi chưa được kích_hoạt bảo_hành nhờ tiki xem giúp',
 nan,
 'z flip4 rất đẹp chỉ nhanh hết pin growing heart',
 'điện_thoại z flip 4 nguyên đai nguyên_kiện của samsung',
 nan,
 'mua cai điện_thoại không tặng nổi_cục xạc pin nữq',
 'giá khá tốt so với thị_trường',
 'máy rát đẹp được tiki giao rất yên_tâm',
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 na

In [ ]:
# Sử dụng hàm map với numpy.vectorize để ánh xạ labels
map_func = np.vectorize(lambda x: labels_dict.get(x, -1))  # Đánh dấu giá trị không tìm thấy với -1
labels = map_func(labels)

In [ ]:
labels

In [ ]:
sentences = np.array(sentences)


In [ ]:
print(labels.shape)
print(sentences.shape)

In [ ]:
df = pd.DataFrame({
    'sentences': sentences,
    'labels': labels
})
df.head()

In [ ]:
# Loại bỏ dòng có giá trị labels là -1
df = df[df['labels'] != -1]
df = df[df['sentences'].astype(str) != 'nan']

In [ ]:
df.shape

## Use TF-IDF for Bayes